# IMPORTS

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import precision_score, recall_score
# from sklearn.metrics import confusion_matrix
import my_pickle as mp
from my_resample import div_count_pos_neg, undersample, oversample
import my_features as mf

# UNPICKLE

In [4]:
# user_df = mp.unjson_it('data_users')
# convo_df = mp.unjson_it('data_convo')

# print("Make sure we're not useing ECT data")
# print(convo_df.timestamp.max())

Make sure we're not useing ECT data
2017-10-02 01:35:58.644000


In [2]:
X = mp.unjson_it('data_features')
print("Make sure we're not useing ECT data")
print(X.timestamp.max())

Make sure we're not useing ECT data
2017-10-02 01:35:58.644000


# GET X AND y

In [3]:
y = X.response
X = X.drop(['response', 'timestamp'], axis=1)
print(len(X))
print(len(y))#.sum()

10937
10937


In [4]:
X.describe()

,I_count_receiver,I_count_sender,I_ratio_receiver,I_ratio_sender,age_dif,age_receiver,age_sender,amenities_overlap,exclaim_count_receiver,exclaim_count_sender,...,sentence_count_receiver,sentence_count_sender,sentence_ratio_receiver,sentence_ratio_sender,smokingOk_receiver,smokingOk_sender,term_receiver,term_sender,type_receiver,type_sender
count,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,...,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000
mean,1.330072,2.049465,28.027685,17.232078,5.391606,26.663802,26.900613,0.651275,0.233428,0.347536,...,1.824723,2.753132,28.029930,17.235737,0.479656,0.578038,17.615704,16.766844,13.757217,16.145979
std,2.892813,3.553003,30.796370,28.661532,6.939003,6.328276,7.227309,1.037781,0.824764,0.954910,...,3.692865,4.412519,30.794329,28.659357,0.499609,0.493895,12.839475,13.571254,19.352382,22.909026
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,12.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.011792,0.007663,2.000000,22.000000,22.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.016260,0.012605,0.000000,0.000000,12.000000,12.000000,1.000000,1.000000
50%,0.000000,1.000000,27.058934,0.018349,4.000000,26.000000,26.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,27.058934,0.020979,0.000000,1.000000,12.000000,12.000000,1.000000,1.000000
75%,1.000000,3.000000,48.393939,34.157705,7.000000,30.000000,30.000000,1.000000,0.000000,0.000000,...,3.000000,4.000000,48.393939,34.157705,1.000000,1.000000,24.000000,20.308388,29.435340,33.873537
max,38.000000,38.000000,340.466910,311.311057,98.000000,118.000000,118.000000,6.000000,20.000000,20.000000,...,93.000000,93.000000,340.466910,311.311057,1.000000,1.000000,154.603746,150.057776,161.439289,164.027541


# OVERSAMPLE

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

X_train, y_train = oversample(X_train, y_train, .5)

# X_ove, y_ove = oversample(X.as_matrix(), y.as_matrix(), .5)

# X_train, X_test, y_train, y_test = train_test_split(X_ove, y_ove, random_state=17)

print("messages sent: {}".format(len(y_train)))
print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}...".format(model.predict(X_test)[:20]))
print("Actual:       {}...".format(y_test[:20]))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

messages sent: 14632
responses:     7316

Prediction:   [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]...
Actual:       [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]...
Model recall: 0.6641509433962264
Model precision: 0.9943502824858758
Score of model: 0.9670932358318098
Score to beat: 0.903107861060329


In [ ]:
messages sent: 14708
responses:     7354

Prediction:   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]...
Actual:       [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]...
Model recall: 0.31683168316831684
Model precision: 0.5052631578947369
Score of model: 0.889945155393053
Score to beat: 0.8892138939670933

# BASELINE RANDOM FOREST

In [ ]:
print("Messages with responses:    {}".format((convo_df.response == True).sum()))
print("Messages without responses: {}".format((convo_df.response == False).sum()))
print("Total messages:             {}".format(len(convo_df)))

In [ ]:
# X_unsamp, y_unsamp = X.as_matrix(), y.as_matrix()

# X_train, X_test, y_train, y_test = train_test_split(X_unsamp, y_unsamp, random_state=17)

X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

X_train, y_train = oversample(X_train, y_train, .5)

print("messages sent: {}".format(len(y_train)))
print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}...".format(model.predict(X_test)[:20]))
print("Actual:       {}...".format(y_test[:20]))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

# UNDERSAMPLE

In [18]:
# X_und, y_und = undersample(X, y, .5)

# X_train, X_test, y_train, y_test = train_test_split(X_und, y_und, random_state=17)

X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

X_train, y_train = oversample(X_train, y_train, .5)

print("messages sent: {}".format(len(y_train)))
print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}...".format(model.predict(X_test)[:20]))
print("Actual:       {}...".format(y_test[:20]))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

messages sent: 14708
responses:     7354

Prediction:   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]...
Actual:       [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]...
Model recall: 0.33663366336633666
Model precision: 0.53125
Score of model: 0.8936014625228519
Score to beat: 0.8892138939670933


# OTHER MODELS KEPT HERE

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

In [ ]:
model = GradientBoostingClassifier()

model.fit(X_train, y_train)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))